In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
## Load the imdb dataset

max_features=10000 ##vocabulary size

# Download the dataset manually
import requests
url = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz'
response = requests.get(url)
with open('imdb.npz', 'wb') as f:
	f.write(response.content)

# Load the dataset
with np.load('imdb.npz', allow_pickle=True) as data:
	X_train, y_train = data['x_train'], data['y_train']
	X_test, y_test = data['x_test'], data['y_test']

# Print the shape of the data
print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_train.shape}, Testing labels shape: {y_test.shape}')

Training data shape: (25000,), Training labels shape: (25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [3]:
maxlen = 500
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)


In [4]:
## Train our simple RNN model

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(128, activation='relu', dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


2025-01-28 11:04:11.615325: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-01-28 11:04:11.615350: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-01-28 11:04:11.615355: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-01-28 11:04:11.615386: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-28 11:04:11.615400: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, 
          validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10


2025-01-28 11:04:12.209575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  4/625 [..............................] - ETA: 17:58:22 - loss: 0.6895 - acc: 0.5703